In [2]:
# ！pip install functorch
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import pandas as pd
import scipy
from scipy import stats
import numpy as np
from os import path

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [34]:
MA_DAYS = 25
trading_days_in_year = 262

# Import raw data from yahoo finance

In [7]:
from google.colab import drive
drive.mount('/content/drive')
data_files_path_prefix = "/content/drive/MyDrive"
data_files_path = "ML-Portfolio-Data"
data_files_path = path.join(data_files_path_prefix, data_files_path)

high_risk_file = 'SPY.csv'
low_risk_file = 'O9P.SI.csv'
high_risk = pd.read_csv(path.join(data_files_path, high_risk_file))
low_risk = pd.read_csv(path.join(data_files_path, low_risk_file))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read files from the same directory
#high_risk = pd.read_csv('SPY.csv')
#low_risk = pd.read_csv('O9P.SI.csv')

In [8]:
high_risk = high_risk[:1008]
low_risk = low_risk[:1008]
print(high_risk.shape)
print(low_risk.shape)

(1008, 7)
(1008, 7)


In [9]:
high_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-08-28,245.169998,245.199997,244.089996,244.570007,224.240860,40565600
1,2017-08-29,243.059998,245.149994,242.929993,244.850006,224.497574,51135700
2,2017-08-30,244.830002,246.320007,244.619995,246.009995,225.561142,62030800
3,2017-08-31,246.720001,247.770004,246.050003,247.490005,226.918106,103803900
4,2017-09-01,247.919998,248.330002,247.669998,247.839996,227.239059,62007000


In [10]:
low_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-08-28,10.90,10.90,10.88,10.88,10.88,7400
1,2017-08-29,10.89,10.90,10.89,10.90,10.90,1400
2,2017-08-30,10.94,10.94,10.94,10.94,10.94,5300
3,2017-08-31,10.92,10.96,10.92,10.93,10.93,30700
4,2017-09-04,10.89,10.89,10.80,10.83,10.83,59800


# ML Portfolio

## Enrich data

### Calculate daily returns

In [11]:
def add_daily_return(market_data):
    market_data["Daily Return"]  = market_data['Close'] - market_data['Open']

add_daily_return(high_risk)
add_daily_return(low_risk)

### Calculate moving average (MA) of daily returns

In [12]:
def add_moving_average(market_data, ma_days):
    temp_vars = []

    # df = market_data
    for i in range(0,ma_days):
        temp_var = "M_{0}".format(i)
        market_data[temp_var] = market_data["Daily Return"].shift(i)
        temp_vars.append(temp_var)

    market_data["MA"] = market_data[temp_vars].mean(axis=1)

    for i in range(0,ma_days):
        temp_var = "M_{0}".format(i)
        market_data.drop(temp_var, axis = 1, inplace = True)

add_moving_average(high_risk, MA_DAYS)
add_moving_average(low_risk, MA_DAYS)


In [ ]:
high_risk.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA
1003,2021-08-23,445.160004,448.230011,443.440002,447.260010,440.990021,54973000,2.100006,0.880400
1004,2021-08-24,447.970001,448.540009,447.420013,447.970001,441.690033,38744700,0.000000,0.665200
1005,2021-08-25,448.170013,449.459991,447.769989,448.910004,442.616913,40529700,0.739991,0.606400
1006,2021-08-26,448.609985,448.859985,446.160004,446.260010,440.004089,57829600,-2.349975,0.483601
1007,2021-08-27,447.119995,450.649994,447.059998,450.250000,443.938110,77235100,3.130005,0.512001


In [13]:
low_risk.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA
1003,2021-08-24,9.83,9.86,9.81,9.84,9.84,19960,0.01,-0.0316
1004,2021-08-25,9.90,9.90,9.85,9.86,9.86,102205,-0.04,-0.0320
1005,2021-08-26,9.89,9.89,9.86,9.86,9.86,66520,-0.03,-0.0296
1006,2021-08-27,9.90,9.90,9.86,9.88,9.88,68015,-0.02,-0.0308
1007,2021-08-30,9.79,9.79,9.77,9.78,9.78,1017465,-0.01,-0.0312


### Calculate ROE

In [14]:
def add_roe(market_data):    
    market_data["Prev Close"] = market_data["Close"].shift(1)
    market_data["ROE"] = (market_data["Close"] - market_data["Prev Close"]) / market_data['Prev Close']

add_roe(high_risk)
add_roe(low_risk)

In [15]:
def add_roe_binary(market_data, tau=-0.005):    
    market_data["ROE Binary"] = np.where(market_data["ROE"].values < tau, 0, 1)

add_roe_binary(high_risk)
add_roe_binary(low_risk)

In [16]:
high_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Prev Close,ROE,ROE Binary
0,2017-08-28,245.169998,245.199997,244.089996,244.570007,224.240860,40565600,-0.599991,-0.599991,NaN,NaN,1
1,2017-08-29,243.059998,245.149994,242.929993,244.850006,224.497574,51135700,1.790008,0.595009,244.570007,0.001145,1
2,2017-08-30,244.830002,246.320007,244.619995,246.009995,225.561142,62030800,1.179993,0.790003,244.850006,0.004738,1
3,2017-08-31,246.720001,247.770004,246.050003,247.490005,226.918106,103803900,0.770004,0.785004,246.009995,0.006016,1
4,2017-09-01,247.919998,248.330002,247.669998,247.839996,227.239059,62007000,-0.080002,0.612002,247.490005,0.001414,1


In [17]:
low_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Prev Close,ROE,ROE Binary
0,2017-08-28,10.90,10.90,10.88,10.88,10.88,7400,-0.02,-0.020000,NaN,NaN,1
1,2017-08-29,10.89,10.90,10.89,10.90,10.90,1400,0.01,-0.005000,10.88,0.001838,1
2,2017-08-30,10.94,10.94,10.94,10.94,10.94,5300,0.00,-0.003333,10.90,0.003670,1
3,2017-08-31,10.92,10.96,10.92,10.93,10.93,30700,0.01,0.000000,10.94,-0.000914,1
4,2017-09-04,10.89,10.89,10.80,10.83,10.83,59800,-0.06,-0.012000,10.93,-0.009149,0


## Build feature space

In [18]:
def remove_for_ma(market_data, ma_days):
  return market_data[ma_days:]

high_risk = remove_for_ma(high_risk, MA_DAYS)
low_risk = remove_for_ma(low_risk, MA_DAYS)

In [19]:
print(high_risk.shape)

(983, 12)


In [20]:
def standardize_columns(market_data, columns):
  for column in columns:
    market_data[column] = market_data[column]/market_data[column].std()

standardize_columns(high_risk, ['Volume', 'Daily Return', 'MA'])
standardize_columns(low_risk, ['Volume', 'Daily Return', 'MA'])

In [21]:
high_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Prev Close,ROE,ROE Binary
25,2017-10-03,252.320007,252.889999,252.229996,252.860001,232.992355,1.367891,0.208144,0.708710,252.320007,0.002140,1
26,2017-10-04,252.690002,253.440002,252.559998,253.160004,233.268799,1.145610,0.181166,0.583476,252.860001,0.001186,1
27,2017-10-05,253.539993,254.679993,253.199997,254.660004,234.650909,1.300584,0.431716,0.577785,253.160004,0.005925,1
28,2017-10-06,254.149994,254.699997,253.850006,254.369995,234.383713,1.651169,0.084801,0.525604,254.660004,-0.001139,1
29,2017-10-09,254.630005,254.699997,253.649994,253.949997,233.996689,0.733043,-0.262114,0.468679,254.369995,-0.001651,1


In [22]:
low_risk.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Prev Close,ROE,ROE Binary
25,2017-10-03,10.86,10.87,10.86,10.86,10.86,0.659605,0.000000,-0.383962,10.85,0.000922,1
26,2017-10-04,10.86,10.86,10.86,10.86,10.86,0.000000,0.000000,-0.393561,10.86,0.000000,1
27,2017-10-05,10.89,10.89,10.88,10.88,10.88,0.172782,-0.045821,-0.403161,10.86,0.001842,1
28,2017-10-06,10.89,10.89,10.87,10.88,10.88,0.371247,-0.045821,-0.422359,10.88,0.000000,1
29,2017-10-09,10.88,10.88,10.85,10.88,10.88,0.254503,0.000000,-0.364764,10.88,0.000000,1


In [23]:
def to_X_train_features(low_risk, high_risk):
  high_risk_train = high_risk[:1000]
  low_risk_train = low_risk[:1000]
  return np.vstack((high_risk_train['Daily Return'], high_risk_train['MA'], high_risk_train['Volume'], low_risk_train['Daily Return'], low_risk_train['MA'], low_risk_train['Volume']))

X = to_X_train_features(low_risk, high_risk).T
print(X.shape, X)

(983, 6) [[ 0.20814449  0.7087102   1.36789127  0.         -0.38396238  0.65960516]
 [ 0.18116558  0.58347574  1.14561011  0.         -0.39356144  0.        ]
 [ 0.43171613  0.57778499  1.30058409 -0.04582138 -0.4031605   0.17278153]
 ...
 [ 0.28523475  1.43829291  0.82981674 -0.13746413 -0.71033041  0.77658292]
 [-0.90581443  1.1470315   1.18401988 -0.09164275 -0.73912759  0.79403619]
 [ 1.20648249  1.2143914   1.58133367 -0.04582138 -0.74872665 11.87832143]]


In [24]:
def to_Y_train_features(low_risk, high_risk):
  high_risk_train = high_risk[0:1000]
  return high_risk_train['ROE Binary'].values

Y = to_Y_train_features(low_risk, high_risk)
print(Y.shape, Y)

(983,) [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 1 0
 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 0 1 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0
 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 1 1 1 1 1 0
 1 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1
 0 1 1 1 1 1 1 1 1

## Build graph

In [25]:
X_tensor = torch.from_numpy(X)
print(type(X), X.shape, X_tensor.size())
Y_tensor = torch.from_numpy(Y).double()
print(type(Y), Y.shape, Y_tensor.size())

<class 'numpy.ndarray'> (983, 6) torch.Size([983, 6])
<class 'numpy.ndarray'> (983,) torch.Size([983])


In [26]:
a = torch.randn((6), requires_grad=True, dtype=torch.double)
b = torch.randn((6), requires_grad=True, dtype=torch.double)
print(a, a.size(), b, b.size())

tensor([ 0.5977, -0.4652,  0.2527, -1.6584, -0.9406,  0.0486],
       dtype=torch.float64, requires_grad=True) torch.Size([6]) tensor([-1.9688, -0.5155,  0.4751, -0.7596,  0.2507,  0.4231],
       dtype=torch.float64, requires_grad=True) torch.Size([6])


In [27]:
lr = 1e-1
n_epochs = 500
torch.manual_seed(42)
lambda1 = 1e-3 #0.5
lambda2 = 1e-3 #0.5

loss_fn = nn.BCELoss()
optimizer = optim.SGD([a, b], lr=lr)

In [28]:
for epoch in range(n_epochs):
  
    yhat = torch.exp(torch.matmul(X_tensor, a)) / (torch.exp (torch.matmul(X_tensor, a)) + torch.exp(torch.matmul(X_tensor, b)))

    loss = loss_fn(yhat, Y_tensor)
    loss.backward()   

    if epoch % 10 == 0:
      print(f"Epoch: {epoch}. Loss: {loss}")

    optimizer.step()
    optimizer.zero_grad()
    
print(a, b)

Epoch: 0. Loss: 0.6592477170372365
Epoch: 10. Loss: 0.47120023377248194
Epoch: 20. Loss: 0.43465862867696153
Epoch: 30. Loss: 0.42363747617787906
Epoch: 40. Loss: 0.4178344013399194
Epoch: 50. Loss: 0.41340467502858896
Epoch: 60. Loss: 0.4095647226881732
Epoch: 70. Loss: 0.40615289110784175
Epoch: 80. Loss: 0.40314812819748197
Epoch: 90. Loss: 0.4005598692056115
Epoch: 100. Loss: 0.39839307383707595
Epoch: 110. Loss: 0.396634419578538
Epoch: 120. Loss: 0.3952499588880977
Epoch: 130. Loss: 0.3941901106778992
Epoch: 140. Loss: 0.39339780367711785
Epoch: 150. Loss: 0.3928162712144505
Epoch: 160. Loss: 0.3923945395399252
Epoch: 170. Loss: 0.39209018750830277
Epoch: 180. Loss: 0.3918699056302546
Epoch: 190. Loss: 0.3917086909106946
Epoch: 200. Loss: 0.3915884205986386
Epoch: 210. Loss: 0.3914963085014207
Epoch: 220. Loss: 0.39142351582671797
Epoch: 230. Loss: 0.3913640227722101
Epoch: 240. Loss: 0.39131377050411037
Epoch: 250. Loss: 0.3912700379509145
Epoch: 260. Loss: 0.39123100383882153
E

## Build efficient frontier

In [29]:
with torch.no_grad():
  y_test = torch.exp(torch.matmul(X_tensor, a)) / (torch.exp (torch.matmul(X_tensor, a)) + torch.exp(torch.matmul(X_tensor, b)))
  print(y_test)

tensor([8.8869e-01, 8.2149e-01, 9.1185e-01, 8.6223e-01, 5.5711e-01, 7.2401e-01,
        8.0939e-01, 7.4679e-01, 7.3119e-01, 7.3294e-01, 8.8141e-01, 6.4151e-01,
        8.8367e-01, 8.7304e-01, 4.8700e-01, 7.8756e-01, 7.2592e-01, 6.9552e-01,
        9.3401e-01, 7.8089e-01, 7.0121e-01, 6.2257e-01, 7.6011e-01, 8.4673e-01,
        8.0370e-01, 6.9437e-01, 8.0630e-01, 8.8200e-01, 8.1306e-01, 8.6986e-01,
        8.0853e-01, 7.5239e-01, 8.9742e-01, 7.4033e-01, 8.7413e-01, 8.9467e-01,
        6.7541e-01, 6.4785e-01, 6.8868e-01, 9.7522e-01, 7.3943e-01, 9.6820e-01,
        9.2887e-01, 4.1184e-01, 8.1581e-01, 7.7555e-01, 7.9379e-01, 8.6409e-01,
        8.0814e-01, 7.5751e-01, 8.0312e-01, 5.4009e-01, 9.6800e-01, 8.0866e-01,
        4.0574e-01, 5.3134e-01, 7.6104e-01, 7.3392e-01, 6.8984e-01, 6.8919e-01,
        7.6142e-01, 5.1307e-01, 9.2609e-01, 9.3885e-01, 8.5677e-01, 9.1113e-01,
        7.9976e-01, 7.2501e-01, 7.8919e-01, 9.2353e-01, 9.4749e-01, 3.7209e-01,
        9.7053e-01, 7.8550e-01, 9.4652e-

### Build ML Portfolio

In [30]:
k = 0.5
calculate_ml_portfolio_weights_lambda = lambda x: 0 if x < k else 1
calculate_ml_portfolio_weights = np.vectorize(calculate_ml_portfolio_weights_lambda)
# vfunc(x)
# calculate_ml_portfolio_weights = functorch.vmap(ml_portfolio_weights, out_dims=1)
# forecast = 
# portfolio_weights = calculate_ml_portfolio_weights(y_test.numpy())
# print(portfolio_weights)

portfolio_weights = y_test.apply_(calculate_ml_portfolio_weights_lambda)
print(portfolio_weights)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1.,
        1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 

# MV Portfolio

In [32]:
def add_daily_return(market_data):
    market_data["Pct Return"]  = market_data['Close'].pct_change()

add_daily_return(high_risk)
add_daily_return(low_risk)

In [54]:
high_risk

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA,Prev Close,ROE,ROE Binary,Pct Return
25,2017-10-03,252.320007,252.889999,252.229996,252.860001,232.992355,1.367891,0.208144,0.708710,252.320007,0.002140,1,NaN
26,2017-10-04,252.690002,253.440002,252.559998,253.160004,233.268799,1.145610,0.181166,0.583476,252.860001,0.001186,1,0.001186
27,2017-10-05,253.539993,254.679993,253.199997,254.660004,234.650909,1.300584,0.431716,0.577785,253.160004,0.005925,1,0.005925
28,2017-10-06,254.149994,254.699997,253.850006,254.369995,234.383713,1.651169,0.084801,0.525604,254.660004,-0.001139,1,-0.001139
29,2017-10-09,254.630005,254.699997,253.649994,253.949997,233.996689,0.733043,-0.262114,0.468679,254.369995,-0.001651,1,-0.001651
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,2021-08-23,445.160004,448.230011,443.440002,447.260010,440.990021,1.125533,0.809462,2.088182,443.359985,0.008797,1,0.008797
1004,2021-08-24,447.970001,448.540009,447.420013,447.970001,441.690033,0.793270,0.000000,1.577758,447.260010,0.001587,1,0.001587
1005,2021-08-25,448.170013,449.459991,447.769989,448.910004,442.616913,0.829817,0.285235,1.438293,447.970001,0.002098,1,0.002098
1006,2021-08-26,448.609985,448.859985,446.160004,446.260010,440.004089,1.184020,-0.905814,1.147032,448.910004,-0.005903,0,-0.005903


In [37]:
high_risk_return_annual = high_risk["Pct Return"].mean() * trading_days_in_year
low_risk_return_annual = low_risk["Pct Return"].mean() * trading_days_in_year
print(high_risk_return_annual)
print(low_risk_return_annual)

0.17665527406579903
-0.019512992661614405


In [38]:
high_risk_var_daily = high_risk["Pct Return"].var()
low_risk_var_daily = low_risk["Pct Return"].var()
print(high_risk_var_daily)
print(low_risk_var_daily)

0.00017197969163255594
6.325232453016195e-05


## Build data for high and low risk

In [44]:
mv_data = pd.DataFrame(data={'high': high_risk['Close'], 'low':low_risk['Close']})
mv_data

,high,low
25,252.860001,10.86
26,253.160004,10.86
27,254.660004,10.88
28,254.369995,10.88
29,253.949997,10.88
...,...,...
1003,447.260010,9.84
1004,447.970001,9.86
1005,448.910004,9.86
1006,446.260010,9.88


In [46]:
def get_annual_sample_return_and_covariance(data):
    daily_return = data.pct_change()
    annual_return = daily_return.mean() * trading_days_in_year
    daily_covariance = data.cov()
    annual_covariance = daily_covariance * trading_days_in_year
    return annual_return, annual_covariance

In [50]:
r, cov = get_annual_sample_return_and_covariance(mv_data)
display(r)
display(cov)

high    0.176655
low    -0.019513
dtype: float64

,high,low
high,719700.556321,-121.568379
low,-121.568379,36.239788


In [51]:
#function obtains minimal risk portfolio 

from scipy import optimize 

def MinimizeRisk(CovarReturns, PortfolioSize):
    
    def  f(x, CovarReturns):
        func = np.matmul(np.matmul(x, CovarReturns), x.T) 
        return func

    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    xinit=np.repeat(0.1, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])

    opt = optimize.minimize (f, x0 = xinit, args = (CovarReturns),  bounds = bnds, \
                             constraints = cons, tol = 10**-3)
    
    return opt

In [52]:
def print_min_variance_portfolio(mean_returns, cov_returns):
    number_of_assets = len(mean_returns)
    result = MinimizeRisk(cov_returns, number_of_assets)

    print()
    minRiskWeights = result.x
    minRiskExpPortfolioReturn = np.matmul(mean_returns.T, minRiskWeights)
    print("Expected Return of Minimum Risk Portfolio:  %7.6f" % minRiskExpPortfolioReturn)
    minRisk = np.matmul(np.matmul(minRiskWeights, cov_returns), minRiskWeights.T) 
    print("Variance of Minimum Risk Portfolio : %7.6f" % minRisk)
    print("S.D. of Minimum Risk Portfolio : %7.6f" % np.sqrt(minRisk))
    threshold = 1e-3
    print("Weights (showing only those > %.6f): " % threshold)
    for i in range(0, number_of_assets):
        if result.x[i] > threshold:
            print(f"{mean_returns.index[i]}\t{result.x[i]:.6f}")
    print('Assets Considered:')
    print(mean_returns.index.to_numpy())

In [53]:
print_min_variance_portfolio(r, cov)


Expected Return of Minimum Risk Portfolio:  0.015714
Variance of Minimum Risk Portfolio : 7194.936594
S.D. of Minimum Risk Portfolio : 84.822972
Weights (showing only those > 0.001000): 
high	0.100000
low	0.100000
Assets Considered:
['high' 'low']
